<a href="https://colab.research.google.com/github/georgia-tech-db/evadb/blob/1063-add-a-model-forecasting-notebook-in-tutorials/tutorials/16-homesale-forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup
We first setup the backend postgres database and EvaDB to bring AI inside database systems.

### Start Postgres

In [1]:
!apt install postgresql
!service postgresql start

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libcommon-sense-perl libjson-perl libjson-xs-perl libtypes-serialiser-perl
  logrotate netbase postgresql-14 postgresql-client-14
  postgresql-client-common postgresql-common ssl-cert sysstat
Suggested packages:
  bsd-mailx | mailx postgresql-doc postgresql-doc-14 isag
The following NEW packages will be installed:
  libcommon-sense-perl libjson-perl libjson-xs-perl libtypes-serialiser-perl
  logrotate netbase postgresql postgresql-14 postgresql-client-14
  postgresql-client-common postgresql-common ssl-cert sysstat
0 upgraded, 13 newly installed, 0 to remove and 16 not upgraded.
Need to get 18.3 MB of archives.
After this operation, 51.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 logrotate amd64 3.19.0-1ubuntu1.1 [54.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main am

### Create User and Database

In [2]:
!sudo -u postgres psql -c "CREATE USER eva WITH SUPERUSER PASSWORD 'password'"
!sudo -u postgres psql -c "CREATE DATABASE evadb"

CREATE ROLE
CREATE DATABASE


###Prettify  Output

In [3]:
import warnings
warnings.filterwarnings("ignore")

from IPython.core.display import display, HTML
def pretty_print(df):
    return display(HTML( df.to_html().replace("\\n","<br>")))

### Install EvaDB

We install EvaDB with extra postgres and forecasting dependency.

In [4]:
%pip install --quiet "evadb[postgres,forecasting] @ git+https://github.com/georgia-tech-db/evadb.git@a40c72ed6cb18993e2ae5bda28c7195f4de4f109"

import evadb
cursor = evadb.connect().cursor()

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.9/110.9 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.0/275.0 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.2/169.2 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.6/154.6 kB 1

Downloading: "http://ml.cs.tsinghua.edu.cn/~chenxi/pytorch-models/mnist-b07bb66b.pth" to /root/.cache/torch/hub/checkpoints/mnist-b07bb66b.pth
100%|██████████| 1.03M/1.03M [00:01<00:00, 1.05MB/s]
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


## Prepare Data
We then prepara the dataset used in this time serise forecasting use case.

### Create Data Source in EvaDB
We use data source to connect EvaDB directly to underlying database systems like Postgres.

In [5]:
params = {
    "user": "eva",
    "password": "password",
    "host": "localhost",
    "port": "5432",
    "database": "evadb",
}
query = f"CREATE DATABASE postgres_data WITH ENGINE = 'postgres', PARAMETERS = {params};"
cursor.query(query).df()

,0
0,The database postgres_data has been successful...


### Load the Datasets
We load the [House Property Sales Time Series](https://www.kaggle.com/datasets/htagholdings/property-sales?resource=download) into our PostgreSQL database.

In [6]:
!mkdir -p content
!wget -qnc -O /content/home_sales.csv https://www.dropbox.com/scl/fi/ww9qejjd3u9gc0m7dagiz/home_sales.csv?rlkey=3gy7yo3michjyumnhi8z24pys&dl=0

In [7]:
cursor.query("""
  USE postgres_data {
    CREATE TABLE IF NOT EXISTS home_sales (saledate VARCHAR(64), MA INT, type VARCHAR(64), bedrooms INT)
  }
""").df()

,status
0,success


In [8]:
cursor.query("""
  USE postgres_data {
    COPY home_sales(saledate, MA, type, bedrooms)
    FROM '/content/home_sales.csv'
    DELIMITER ',' CSV HEADER
  }
""").df()

,status
0,success


### Preview the Data
The `home_sales` table contains 4 columns.
- saledate: the date that home was sold
- ma: moving average of the historical median price of the home
- type: whether the home is house or unit
- bedrooms: number of bedrooms

In [9]:
cursor.query("SELECT * FROM postgres_data.home_sales LIMIT 3;").df()

,home_sales.ma,home_sales.bedrooms,home_sales.saledate,home_sales.type
0,441854,2,30/09/2007,house
1,441854,2,31/12/2007,house
2,441854,2,31/03/2008,house


## Analysis Data with EvaDB

We then use EvaDB to train a model to forecast the home price.

### Train the Forecast Model
We use the [statsforecast](https://github.com/Nixtla/statsforecast) engine to train a time serise forecast model for sale prices of home with two bedrooms.

In [10]:
cursor.query("""
  CREATE FUNCTION IF NOT EXISTS HomeSaleForecast FROM
    (
      SELECT type, saledate, ma
      FROM postgres_data.home_sales
      WHERE bedrooms = 2
    )
  TYPE Forecasting
  PREDICT 'ma'
  TIME 'saledate'
  ID 'type'
  FREQUENCY 'M';
""").df()

,0
0,Function HomeSaleForecast successfully added t...


### Use the Forecast Model
We then use the `HomeSaleForecast` model to predict the sale price for homes with two bedrooms for the next three month.

In [11]:
cursor.query("SELECT HomeSaleForecast(3);").df()

,homesaleforecast.type,homesaleforecast.saledate,homesaleforecast.ma
0,house,2019-10-31,510712.00000
1,house,2019-11-30,510712.00000
2,house,2019-12-31,510712.00000
3,unit,2019-10-31,423431.37500
4,unit,2019-11-30,422450.78125
5,unit,2019-12-31,421470.15625


We can use `ORDER BY` to find out the type of home and months that have lower market price.

In [15]:
cursor.query("SELECT HomeSaleForecast(3) ORDER BY ma;").df()

09-14-2023 06:17:03 ERROR [statement_binder_context:statement_binder_context.py:raise_error:0147] Found invalid column ma
ERROR:evadb.utils.logging_manager:Found invalid column ma


BinderError: ignored